In [2]:
import xarray as xr
import pathlib as pl
import pyproj as pp
import subprocess
import sys
import random

In [3]:
# define working directories for use in making a forward run of SWB2
base_dir = str(pl.Path.cwd().parent)
python_script_dir = pl.Path.cwd().parent / 'python'
swb_work_dir = 'test__netcdf_daymet'
logfile_dir = 'logfiles'
test_data_dir = pl.Path.cwd().parent / 'test_data'
output_dir = 'output'
swb2_executable = str(pl.Path.cwd().parent / 'bin' /'swb2.exe')
sys.path.append(str(python_script_dir))

import run_management as rm

In [4]:
rm.destroy_model_work_output_and_logfile_dirs(base_dir=base_dir,
                                              swb_work_dir=swb_work_dir,
                                              logfile_dir=logfile_dir,
                                              output_dir=output_dir)
rm.create_model_work_output_and_logfile_dirs(base_dir=base_dir,
                                             swb_work_dir=swb_work_dir,
                                             logfile_dir=logfile_dir,
                                             output_dir=output_dir)

In [5]:
dry_run = False
lookup_dir_arg_text = f"--lookup_dir={str(test_data_dir)}"
weather_data_dir_arg_text = f"--weather_data_dir={str(test_data_dir)}"
data_dir_arg_text = f"--data_dir={str(test_data_dir)}"
output_dir_arg_text = f"--output_dir={str(output_dir)}"
logfile_dir_arg_text = f"--logfile_dir={str(logfile_dir)}"
control_file_path = str(test_data_dir / 'swb_control_file_daymet.ctl')
output_prefix = '--output_prefix=swb_daymet_test_'

swb_arg_text = [swb2_executable, output_dir_arg_text, lookup_dir_arg_text, weather_data_dir_arg_text, 
                data_dir_arg_text, output_prefix, logfile_dir_arg_text, control_file_path]

f = open("stdout.txt", "w")

if not dry_run:
  with rm.cd(pl.Path(base_dir) / swb_work_dir):
#    p = subprocess.Popen(swb_arg_text, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    p = subprocess.Popen(swb_arg_text, stdout=f, stderr=subprocess.DEVNULL)
    p.wait()

In [6]:
# define CRS for Daymet and SWB grids
crs_dm = pp.CRS.from_proj4("+proj=lcc +lat_1=25.0 +lat_2=60.0 +lat_0=42.5 +lon_0=-100.0 +x_0=0.0 +y_0=0.0 +ellps=GRS80 +datum=NAD83 +units=m +no_defs")
crs_swb = pp.CRS.from_epsg(5070)

In [7]:
df_dm = xr.open_dataset(test_data_dir / 'daymet_v4_daily_na_prcp_2000_mn__14-day.nc')
df_swb = xr.open_dataset( pl.Path(base_dir).resolve() / swb_work_dir / output_dir / 
                         "swb_daymet_test_gross_precipitation__2000-01-01_to_2000-01-14__688_by_620.nc")

In [8]:
proj = pp.transformer.Transformer.from_crs(crs_swb, crs_dm)

In [29]:
x = df_swb.x.values
y = df_swb.y.values

In [30]:
#x_indices = randrange(start=20, stop=)
#y_indices = [25, 100, 200, 300, 400, 500, 600]

x1, y1 = proj.transform(x[57],y[42])

In [31]:
df_swb.x[0]

<xarray.DataArray 'x' ()>
array(-124545.)
Coordinates:
    x        float64 -1.245e+05
    crs      int32 ...
Attributes:
    units:          meter
    long_name:      x coordinate of projection
    standard_name:  projection_x_coordinate

In [32]:
data_dm = df_dm.sel(x=x1, y=y1, method='nearest')
data_swb = df_swb.sel(x=x[57],y=y[42], method='nearest')

In [36]:
pdm = data_dm.prcp

In [50]:
pswb = data_swb.gross_precipitation*25.4

In [58]:
pdm - pswb

<xarray.DataArray (time: 0)>
array([], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 
    lat      float32 ...
    lon      float32 ...
    crs      int32 ...